In [2]:
# Install required python packages
!pip3 install --upgrade --user google-cloud-aiplatform

In [3]:
# restart the kernel after libraries are loaded
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
# Define project information
PROJECT_ID = "!(gcloud config get-value core/project)"  # @param {type:"string"}

region_output = !gcloud compute project-info describe --format="value(commonInstanceMetadata.items[google-compute-default-region])"
LOCATION = region_output[0].strip()

# Get project ID properly
project_output = !gcloud config get-value core/project
PROJECT_ID = project_output[0].strip()

print("LOCATION:", LOCATION)
print("PROJECT_ID:", PROJECT_ID)

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [1]:
# Please like share & subscribe to Techcps
# YouTube https://www.youtube.com/@techcps

print("Please like share & subscribe to Techcps https://www.youtube.com/@techcps")

In [2]:
# Task 3
# import software libraries required
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

In [4]:
# Task 3.1
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
# Use the Gemini 1.5 Pro 002 Model
model = GenerativeModel("gemini-1.5-pro-002")

In [5]:
# Task 3.2
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "Location"
            }
        }
    },
)

In [6]:
# Task 3.3
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

In [7]:
# Task 3.4
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling#function-calling-generation-sdk-sample
prompt = "What is the weather like in Boston?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[weather_tool],
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "Boston"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  avg_logprobs: -1.7031977352287087e-08
}
usage_metadata {
  prompt_token_count: 23
  candidates_token_count: 7
  total_token_count: 30
  prompt_tokens_details {
    modality: TEXT
    token_count: 23
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 7
  }
}
model_version: "gemini-1.5-pro-002"
create_time {
  seconds: 1745890073
  nanos: 732072000
}
response_id: "GSsQaKjXLNrghMIPvIO2gAg"

In [8]:
# Task 4
# Run the following cell to import required libraries 
from vertexai.generative_models import (
    GenerationConfig,
    Image,
    Part,
)

In [9]:
# Task 4.1
# Load the correct Gemini model use the following documentation to assist:
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/overview#supported-use-cases
# Use the Gemini 1.5 Pro 002 Model
multimodal_model = GenerativeModel("gemini-1.5-pro-002")

In [10]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_image(content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
        return False
    display_images([content])
    return True


def display_content_as_video(content: str | Image | Part) -> bool:
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=600))
    return True


def print_multimodal_prompt(contents: list[str | Image | Part]):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if display_content_as_image(content):
            continue
        if display_content_as_video(content):
            continue
        print(content)

In [13]:
# Task 4.2 Generate a video description
# In this cell, update the prompt to ask Gemini to describe the video URL referenced.
# You can use the documentation at the following link to assist.
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/sdk-for-gemini/gemini-sdk-overview-reference#generate-content-from-video
# 
# Video URI: gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4
# 
prompt = """
What is shown in this video?
Where should I go to see it?
What are the top 5 places in the world that look like this?
"""
video = Part.from_uri(
    uri="gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents, stream=True)

print("-------Prompt--------")
print_multimodal_prompt(contents)


-------Prompt--------

What is shown in this video?
Where should I go to see it?
What are the top 5 places in the world that look like this?

